<a href="https://colab.research.google.com/github/Jakelinecs/Tareas-Machine-Learning/blob/main/N33.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, GRU, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
import time

# --- Configuración de Parámetros ---

# Límite del vocabulario (solo se consideran las 'max_words' más frecuentes)
MAX_WORDS = 10000
# Longitud máxima de la secuencia (las secuencias más largas se truncan, las más cortas se rellenan)
MAX_LEN = 100
# Dimensiones de la capa de embedding
EMBEDDING_DIM = 32
# Número de nodos recurrentes (unidades)
RNN_UNITS = 32
# Épocas de entrenamiento (se mantiene bajo para una ejecución rápida)
EPOCHS = 3
# Tamaño del lote
BATCH_SIZE = 128

# --- Carga y Preprocesamiento de Datos ---

def load_and_preprocess_data():
    """Carga el conjunto de datos IMDB y lo preprocesa."""
    print("Cargando datos de IMDB...")
    # Carga de datos de entrenamiento y prueba
    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=MAX_WORDS)

    print(f"Número de secuencias de entrenamiento: {len(x_train)}")
    print(f"Número de secuencias de prueba: {len(x_test)}")

    # Relleno (padding) de secuencias para que todas tengan la misma longitud (MAX_LEN)
    # padding='post' asegura que el relleno (ceros) se añada al final de la secuencia
    print("Rellenando secuencias...")
    x_train = pad_sequences(x_train, maxlen=MAX_LEN, padding='post', truncating='post')
    x_test = pad_sequences(x_test, maxlen=MAX_LEN, padding='post', truncating='post')

    return x_train, y_train, x_test, y_test

# --- Definición del Modelo y Entrenamiento ---

def build_and_train_model(rnn_layer_type, rnn_units, x_train, y_train, x_test, y_test):
    """
    Construye, compila y entrena un modelo Keras con la capa RNN especificada.
    """
    print(f"\n--- Entrenando modelo con capa: {rnn_layer_type.__name__} ---")

    # 1. Definición del Modelo
    model = Sequential()

    # Capa de Embedding: Mapea cada índice de palabra a un vector denso de tamaño EMBEDDING_DIM
    model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_LEN))

    # 2. Capa Recurrente
    model.add(rnn_layer_type(rnn_units))

    # 3. Capa de Salida (Clasificación binaria: Positivo o Negativo)
    model.add(Dense(1, activation='sigmoid'))

    # 4. Compilación
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # 5. Entrenamiento
    start_time = time.time()
    history = model.fit(
        x_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(x_test, y_test),
        verbose=1
    )
    end_time = time.time()

    # 6. Evaluación
    loss, acc = model.evaluate(x_test, y_test, verbose=0)

    print(f"Tiempo de entrenamiento: {end_time - start_time:.2f} segundos")
    print(f"Precisión (Accuracy) en datos de prueba: {acc:.4f}")

    return model, history, acc

# --- Función Principal de Comparación ---

def rnn_comparison():
    """Ejecuta la comparación entre SimpleRNN, GRU y LSTM."""

    x_train, y_train, x_test, y_test = load_and_preprocess_data()

    results = {}

    # Lista de capas RNN a comparar (Pregunta 1)
    rnn_layers = [SimpleRNN, GRU, LSTM]

    for rnn_layer in rnn_layers:
        _, _, acc = build_and_train_model(
            rnn_layer,
            RNN_UNITS,
            x_train, y_train, x_test, y_test
        )
        results[rnn_layer.__name__] = acc

    print("\n" + "="*50)
    print("RESUMEN DE PRECISIÓN EN DATOS DE PRUEBA")
    print("="*50)
    for name, accuracy in sorted(results.items(), key=lambda item: item[1], reverse=True):
        print(f"| {name:<12}: {accuracy:.4f} |")
    print("="*50)

    # Demostración adicional de ConvLSTM2D (no comparable directamente)
    print("\n--- Demostración de ConvLSTM2D (requiere datos 5D) ---")
    print("Nota: ConvLSTM2D se utiliza para datos de video/imágenes secuenciales, no texto.")


if __name__ == '__main__':
    # Configurar el nivel de log de TensorFlow para no mostrar mensajes de advertencia excesivos
    tf.get_logger().setLevel('ERROR')
    rnn_comparison()


Cargando datos de IMDB...
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Número de secuencias de entrenamiento: 25000
Número de secuencias de prueba: 25000
Rellenando secuencias...

--- Entrenando modelo con capa: SimpleRNN ---
Epoch 1/3


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


196/196 ━━━━━━━━━━━━━━━━━━━━ 16s 68ms/step - accuracy: 0.5639 - loss: 0.6704 - val_accuracy: 0.7626 - val_loss: 0.5246
Epoch 2/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 17s 50ms/step - accuracy: 0.8208 - loss: 0.4290 - val_accuracy: 0.7904 - val_loss: 0.4506
Epoch 3/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - accuracy: 0.8900 - loss: 0.2878 - val_accuracy: 0.7717 - val_loss: 0.5601
Tiempo de entrenamiento: 42.88 segundos
Precisión (Accuracy) en datos de prueba: 0.7717

--- Entrenando modelo con capa: GRU ---
Epoch 1/3


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


196/196 ━━━━━━━━━━━━━━━━━━━━ 25s 112ms/step - accuracy: 0.5505 - loss: 0.6733 - val_accuracy: 0.7840 - val_loss: 0.4764
Epoch 2/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 42s 117ms/step - accuracy: 0.8490 - loss: 0.3664 - val_accuracy: 0.8087 - val_loss: 0.4327
Epoch 3/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 40s 110ms/step - accuracy: 0.8893 - loss: 0.2858 - val_accuracy: 0.8082 - val_loss: 0.4380
Tiempo de entrenamiento: 106.33 segundos
Precisión (Accuracy) en datos de prueba: 0.8082

--- Entrenando modelo con capa: LSTM ---
Epoch 1/3


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


196/196 ━━━━━━━━━━━━━━━━━━━━ 21s 93ms/step - accuracy: 0.5986 - loss: 0.6437 - val_accuracy: 0.8011 - val_loss: 0.4347
Epoch 2/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 25s 127ms/step - accuracy: 0.8576 - loss: 0.3628 - val_accuracy: 0.8025 - val_loss: 0.4297
Epoch 3/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 19s 95ms/step - accuracy: 0.8898 - loss: 0.2930 - val_accuracy: 0.8077 - val_loss: 0.4702
Tiempo de entrenamiento: 64.82 segundos
Precisión (Accuracy) en datos de prueba: 0.8077

RESUMEN DE PRECISIÓN EN DATOS DE PRUEBA
| GRU         : 0.8082 |
| LSTM        : 0.8077 |
| SimpleRNN   : 0.7717 |

--- Demostración de ConvLSTM2D (requiere datos 5D) ---
Nota: ConvLSTM2D se utiliza para datos de video/imágenes secuenciales, no texto.


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import reuters # ¡Cambiado a Reuters!
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, GRU, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical # Importado para One-Hot Encoding
import time

# --- Configuración de Parámetros ---

# Límite del vocabulario (solo se consideran las 'max_words' más frecuentes)
MAX_WORDS = 10000
# Longitud máxima de la secuencia
MAX_LEN = 100
# Dimensiones de la capa de embedding
EMBEDDING_DIM = 32
# Número de nodos recurrentes (unidades)
RNN_UNITS = 32
# Épocas de entrenamiento (se mantiene bajo para una ejecución rápida)
EPOCHS = 3
# Tamaño del lote
BATCH_SIZE = 128
# Número de clases para Reuters (46 temas)
NUM_CLASSES = 46

# --- Carga y Preprocesamiento de Datos ---

def load_and_preprocess_data():
    """Carga el conjunto de datos Reuters y lo preprocesa para clasificación multiclase."""
    print("Cargando datos de Reuters Newswire...")
    # Carga de datos de entrenamiento y prueba
    (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=MAX_WORDS)

    print(f"Número de secuencias de entrenamiento: {len(x_train)}")
    print(f"Número de secuencias de prueba: {len(x_test)}")
    print(f"Número de clases: {NUM_CLASSES}")

    # Relleno (padding) de secuencias
    print("Rellenando secuencias...")
    x_train = pad_sequences(x_train, maxlen=MAX_LEN, padding='post', truncating='post')
    x_test = pad_sequences(x_test, maxlen=MAX_LEN, padding='post', truncating='post')

    # CONVERSIÓN A ONE-HOT ENCODING (Necesario para 'categorical_crossentropy')
    y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
    y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

    return x_train, y_train, x_test, y_test

# --- Definición del Modelo y Entrenamiento ---

def build_and_train_model(rnn_layer_type, rnn_units, x_train, y_train, x_test, y_test):
    """
    Construye, compila y entrena un modelo Keras con la capa RNN especificada.
    Adaptado para clasificación multiclase (Reuters).
    """
    print(f"\n--- Entrenando modelo con capa: {rnn_layer_type.__name__} ---")

    # 1. Definición del Modelo
    model = Sequential()

    # Capa de Embedding
    model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_LEN))

    # 2. Capa Recurrente
    model.add(rnn_layer_type(rnn_units))

    # 3. Capa de Salida (Clasificación Multiclase: 46 temas)
    # Usando NUM_CLASSES y activación 'softmax'
    model.add(Dense(NUM_CLASSES, activation='softmax'))

    # 4. Compilación
    model.compile(
        optimizer='adam',
        # Usando 'categorical_crossentropy' para clasificación multiclase
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # 5. Entrenamiento
    start_time = time.time()
    history = model.fit(
        x_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(x_test, y_test),
        verbose=1
    )
    end_time = time.time()

    # 6. Evaluación
    loss, acc = model.evaluate(x_test, y_test, verbose=0)

    print(f"Tiempo de entrenamiento: {end_time - start_time:.2f} segundos")
    print(f"Precisión (Accuracy) en datos de prueba: {acc:.4f}")

    return model, history, acc

# --- Función Principal de Comparación ---

def rnn_comparison():
    """Ejecuta la comparación entre SimpleRNN, GRU y LSTM con el dataset Reuters."""

    x_train, y_train, x_test, y_test = load_and_preprocess_data()

    results = {}

    # Lista de capas RNN a comparar (SimpleRNN, GRU, LSTM)
    rnn_layers = [SimpleRNN, GRU, LSTM]

    for rnn_layer in rnn_layers:
        _, _, acc = build_and_train_model(
            rnn_layer,
            RNN_UNITS,
            x_train, y_train, x_test, y_test
        )
        results[rnn_layer.__name__] = acc

    print("\n" + "="*50)
    print("RESUMEN DE PRECISIÓN EN DATOS DE PRUEBA (Reuters)")
    print("="*50)
    for name, accuracy in sorted(results.items(), key=lambda item: item[1], reverse=True):
        print(f"| {name:<12}: {accuracy:.4f} |")
    print("="*50)

    # Demostración adicional de ConvLSTM2D (no comparable directamente)
    print("\n--- Demostración de ConvLSTM2D ---")
    print("ConvLSTM2D requiere un tensor 5D (muestras, tiempo, filas, columnas, canales).")
    print("No es directamente aplicable a este dataset de texto plano.")


if __name__ == '__main__':
    # Configurar el nivel de log de TensorFlow para no mostrar mensajes de advertencia excesivos
    tf.get_logger().setLevel('ERROR')
    rnn_comparison()


Cargando datos de Reuters Newswire...
2110848/2110848 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Número de secuencias de entrenamiento: 8982
Número de secuencias de prueba: 2246
Número de clases: 46
Rellenando secuencias...

--- Entrenando modelo con capa: SimpleRNN ---
Epoch 1/3


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.3231 - loss: 3.4839 - val_accuracy: 0.4555 - val_loss: 2.3689
Epoch 2/3
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.4836 - loss: 2.1686 - val_accuracy: 0.4835 - val_loss: 1.9934
Epoch 3/3
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5374 - loss: 1.8182 - val_accuracy: 0.4902 - val_loss: 1.9907
Tiempo de entrenamiento: 5.41 segundos
Precisión (Accuracy) en datos de prueba: 0.4902

--- Entrenando modelo con capa: GRU ---
Epoch 1/3
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.2752 - loss: 3.4441 - val_accuracy: 0.3695 - val_loss: 2.3766
Epoch 2/3
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.3628 - loss: 2.3270 - val_accuracy: 0.3731 - val_loss: 2.2949
Epoch 3/3
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.3661 - loss: 2.2207 - val_accuracy: 0.3998 - val_loss: 2.1440
Tiempo de entrenamiento: 14.97 segundos
Precisión (Accuracy) en datos de prueba: 0.3998

--- Entrenando modelo con capa: LSTM 